In [ ]:
# Imports
import openpyxl
import os
import win32com.client as win32

In [ ]:
# Define filepaths (duration - 1 min)

excel_name = "Übersicht EStKöSt-Erklärungen und LV.xlsx" # ACHTUNG Excel ohne Makros verwenden! 
excel_path = os.path.join(os.getcwd(), excel_name)

workbook = openpyxl.load_workbook(excel_path)

# Select the source sheet and the results sheet
source_sheet = workbook["KöSt-Erklärungen 2022"]         # Excel sheet aus dem die Daten genommen werden

In [ ]:
# Iterate over rows 4 to 26
for row in range(4, 27):  # 4, 27
    # Check if the value in column F is "offen"
    if source_sheet.cell(row=row, column=6).value == "offen":
        # Get the name, gender, and email from columns J, K, and L
        client = source_sheet.cell(row=row, column=2).value
        gender = source_sheet.cell(row=row, column=11).value
        name = source_sheet.cell(row=row, column=10).value
        email = source_sheet.cell(row=row, column=12).value
        # email_tl = "marta.primethofer@icon.at" 
        email_cc = source_sheet.cell(row=row, column=13).value # + "" + email_tl
        
        # Get the tax year from cell B2
        tax_year = source_sheet.cell(row=2, column=2).value
        
        # Determine Covid extension for the current year
        if int(tax_year) == 2022:
            covid_extension = "31. Juli 2024"
        if int(tax_year) == 2023:
            covid_extension = "31. Mai 2025"
        if int(tax_year) == 2024:
            covid_extension = "30. April 2026"
        
        # Create the email subject
        email_subject = f"KSt {tax_year} DE - {client} - Anfrage Daten"
        
        # Create the email body with the appropriate salutation based on gender
        if gender == "m":
            salutation = f"Sehr geehrter {name},"
        else:
            salutation = f"Sehr geehrte {name},"

        body = f"""
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        {salutation}<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Wir möchten uns gerne in Bezug auf die <b>deutsche Körperschaftsteuererklärung {tax_year}</b> der <b>{client}</b> mit Ihnen abstimmen.<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Um einen möglichst reibungsfreien Ablauf zu gewährleisten, möchten wir Sie bitten uns die <b>Unterlagen</b> für die laufenden bzw. die heuer begonnenen Betriebsstätten in Deutschland zukommen zu lassen.<o:p></o:p></span></p> 
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Die <b>Frist</b> für die Einreichung der deutschen Körperschaftsteuer ist grundsätzlich in beratenen Fällen der letzte Tag des Monats Februar des auf das Steuerjahr zweitfolgenden Jahres (<b>für {tax_year} Ende Februar {int(tax_year)+2}</b>).<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Aufgrund der <b>COVID-bedingten Anpassung</b> der Fristen zur Abgabe von Steuererklärungen durch das deutsche BMF wurde die Frist für <b>{tax_year} bis {covid_extension}</b> verlängert.<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Es ist jedoch den deutschen Steuerbehörden unbenommen vor Fristende eine <b>Aufforderung zur Abgabe der Steuererklärung</b> zu erlassen. Im Herbst/Winter {tax_year} wurde betreffend die Steuererklärungen {int(tax_year)-1} in überdurchschnittlichem Ausmaß davon Gebrauch gemacht.
        Daher regen wir an diese Vorgehensweise der Finanzverwaltung für {tax_year} zu erwarten und entsprechend einzuplanen. Nach Erlass der Aufforderung verbleiben von Gesetzes wegen <b>4 Monate zur Einreichung</b> der Erklärung. 
        <o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>
        Gerne stehen wir vorab für Rückfragen zur Verfügung.<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>Vielen Dank und beste Grüße!<o:p></o:p></span></p>

        """
        
        # Create the email in Outlook using win32

        outlook = win32.Dispatch("outlook.Application")
        mail = outlook.CreateItem(0) 
        mail.Subject = email_subject
        mail.HTMLBody = body
        mail.Sender = "alexander.aufreiter@icon.at"
        mail.To = email
        # mail.CC = email_cc

        #mail.Display()     # opens a separate window for each email
        mail.Save()         # saves the email as draft
        #mail.Send()        # sends the email immediately
        print(f"one mail for {name}")
    else: 
        name = source_sheet.cell(row=row, column=10).value
        print(f"no email for you, {name}")

# Close the workbook
workbook.close()


In [ ]:
print(covid_extension)